# Day 12: Passage Pathing

[https://adventofcode.com/2021/day/12](https://adventofcode.com/2021/day/12)

## Description

### Part One

With your <span title="Sublime.">submarine's subterranean subsystems subsisting suboptimally</span>, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just _a_ path - the only way to know if you've found the _best_ path is to find _all_ of them.

Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:

    start-A
    start-b
    A-c
    A-b
    b-d
    A-end
    b-end


This is a list of how all of the caves are connected. You start in the cave named `start`, and your destination is the cave named `end`. An entry like `b-d` means that cave `b` is connected to cave `d` - that is, you can move between them.

So, the above cave system looks roughly like this:

        start
        /   \
    c--A-----b--d
        \   /
         end


Your goal is to find the number of distinct _paths_ that start at `start`, end at `end`, and don't visit small caves more than once. There are two types of caves: _big_ caves (written in uppercase, like `A`) and _small_ caves (written in lowercase, like `b`). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should _visit small caves at most once_, and can _visit big caves any number of times_.

Given these rules, there are _`10`_ paths through this example cave system:

    start,A,b,A,c,A,end
    start,A,b,A,end
    start,A,b,end
    start,A,c,A,b,A,end
    start,A,c,A,b,end
    start,A,c,A,end
    start,A,end
    start,b,A,c,A,end
    start,b,A,end
    start,b,end


(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)

Note that in this cave system, cave `d` is never visited by any path: to do so, cave `b` would need to be visited twice (once on the way to cave `d` and a second time when returning from cave `d`), and since cave `b` is small, this is not allowed.

Here is a slightly larger example:

    dc-end
    HN-start
    start-kj
    dc-start
    dc-HN
    LN-dc
    HN-end
    kj-sa
    kj-HN
    kj-dc


The `19` paths through it are as follows:

    start,HN,dc,HN,end
    start,HN,dc,HN,kj,HN,end
    start,HN,dc,end
    start,HN,dc,kj,HN,end
    start,HN,end
    start,HN,kj,HN,dc,HN,end
    start,HN,kj,HN,dc,end
    start,HN,kj,HN,end
    start,HN,kj,dc,HN,end
    start,HN,kj,dc,end
    start,dc,HN,end
    start,dc,HN,kj,HN,end
    start,dc,end
    start,dc,kj,HN,end
    start,kj,HN,dc,HN,end
    start,kj,HN,dc,end
    start,kj,HN,end
    start,kj,dc,HN,end
    start,kj,dc,end


Finally, this even larger example has `226` paths through it:

    fs-end
    he-DX
    fs-he
    start-DX
    pj-DX
    end-zg
    zg-sl
    zg-pj
    pj-he
    RW-he
    fs-DX
    pj-RW
    zg-RW
    start-pj
    he-WI
    zg-he
    pj-fs
    start-RW


_How many paths through this cave system are there that visit small caves at most once?_

In [1]:
from collections import defaultdict

def find_path(routes: dict[str, set], seen: set, current: str, small_caves: set[str], small_cave_count: int=0):
    for nxt in routes[current]:
        if nxt in small_caves:
            if nxt == "end":
                yield seen
            elif len(seen | {nxt}) < small_cave_count+1:
                continue  # can't revisit a small room.
            else:
                yield from find_path(routes, seen | {nxt}, nxt, small_caves, small_cave_count+1)
        else:
            yield from find_path(routes, seen, nxt, small_caves, small_cave_count)

def load(filename: str) -> dict[str, set]:
    routes = defaultdict(set)
    for a, b in [line.strip().split("-") for line in open(filename)]:
        # avoid adding any routes back to start
        if b != 'start':
            routes[a].add(b)
        if a != 'start':
            routes[b].add(a)
    # and no routes away from end
    routes['end'] = set()
    return routes

def part1(filename):
    routes = load(filename)

    small_caves = {cave for cave in routes if cave.islower()}
    return len(list(find_path(routes, seen=set(), current='start', small_caves=small_caves, small_cave_count=0)))


assert part1("input/day12-test.txt") == 19
print("part 1", part1("input/day12.txt"))

part 1 3369

<div class="alert alert-info">Confession time: my original code for this was so bad I rewrote it from scratch
(after submitting the answer). This version is much cleaner (the original is in git history if you must). Check to
avoid revisiting a small cave by counting total small caves visited and comparing against the size of a set of them.</div>

**Your puzzle answer was 3369.**

### Part Two

After reviewing the available paths, you realize you might have time to visit a single small cave _twice_. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named `start` and `end` can only be visited _exactly once each_: once you leave the `start` cave, you may not return to it, and once you reach the `end` cave, the path must end immediately.

Now, the `36` possible paths through the first example above are:

    start,A,b,A,b,A,c,A,end
    start,A,b,A,b,A,end
    start,A,b,A,b,end
    start,A,b,A,c,A,b,A,end
    start,A,b,A,c,A,b,end
    start,A,b,A,c,A,c,A,end
    start,A,b,A,c,A,end
    start,A,b,A,end
    start,A,b,d,b,A,c,A,end
    start,A,b,d,b,A,end
    start,A,b,d,b,end
    start,A,b,end
    start,A,c,A,b,A,b,A,end
    start,A,c,A,b,A,b,end
    start,A,c,A,b,A,c,A,end
    start,A,c,A,b,A,end
    start,A,c,A,b,d,b,A,end
    start,A,c,A,b,d,b,end
    start,A,c,A,b,end
    start,A,c,A,c,A,b,A,end
    start,A,c,A,c,A,b,end
    start,A,c,A,c,A,end
    start,A,c,A,end
    start,A,end
    start,b,A,b,A,c,A,end
    start,b,A,b,A,end
    start,b,A,b,end
    start,b,A,c,A,b,A,end
    start,b,A,c,A,b,end
    start,b,A,c,A,c,A,end
    start,b,A,c,A,end
    start,b,A,end
    start,b,d,b,A,c,A,end
    start,b,d,b,A,end
    start,b,d,b,end
    start,b,end


The slightly larger example above now has `103` paths through it, and the even larger example now has `3509` paths through it.

Given these new rules, _how many paths through this cave system are there?_

In [2]:
def part2(filename):
    routes = load(filename)

    small_caves = {cave for cave in routes if cave.islower()}
    return len(list(find_path(routes, seen=set(), current='start', small_caves=small_caves, small_cave_count=-1)))

assert part2("input/day12-test.txt") == 103
print("part 2", part2("input/day12.txt"))

part 2 85883

<div class="alert alert-info">We can allow for revisiting one small cave simply by starting the count one lower.</div>


**Your puzzle answer was 85883.**